In [1]:
import bonesis
import ginsim
import mpbn

from colomoto_jupyter import tabulate
import pandas as pd
from zipfile import ZipFile

In [2]:
exact_pkn = True
maxclause = 32
exclude_cyclic = True
bundle_prefix = "bundle-{}pkn{}-{}".format("exact" if exact_pkn else "any", maxclause,
        "nocyclic-" if exclude_cyclic else "")
limit = 1000
threads = 4
bundle_prefix

'bundle-exactpkn32-nocyclic-'

In [3]:
model = ginsim.load("Master_Model.zginml")
ginsim.show(model)

## Architecture

We take the influence graph of the original model as constraint for the synthesis.
We impose that all identified BNs have the exact same influence graph.
As some nodes have high in-degree, we limit the number of clauses in Boolean functions.

In [4]:
bn = mpbn.MPBooleanNetwork(ginsim.to_minibn(model))

In [5]:
dom = bonesis.InfluenceGraph.from_ginsim(model, exact=exact_pkn, maxclause=maxclause)
#dom = bonesis.domains.BooleanNetwork(bn)

## Input Data

### Initial condition

Our initial states can be have `ECMicroenv` and `DNAdamage` active or not, all other nodes are inactive, except micro-RNAs which are active.

In [6]:
init = bn.zero()
# del free nodes
del init["ECMicroenv"]
del init["DNAdamage"]
# miR?
for miR in ["miR200", "miR203", "miR34"]:
    init[miR] = init[miR]+1 # ensure names exist
init

{'AKT1': 0,
 'AKT2': 0,
 'Apoptosis': 0,
 'CDH1': 0,
 'CDH2': 0,
 'CTNNB1': 0,
 'CellCycleArrest': 0,
 'DKK1': 0,
 'EMT': 0,
 'ERK': 0,
 'GF': 0,
 'Invasion': 0,
 'Metastasis': 0,
 'Migration': 0,
 'NICD': 0,
 'SMAD': 0,
 'SNAI1': 0,
 'SNAI2': 0,
 'TGFbeta': 0,
 'TWIST1': 0,
 'VIM': 0,
 'ZEB1': 0,
 'ZEB2': 0,
 'miR200': 1,
 'miR203': 1,
 'miR34': 1,
 'p21': 0,
 'p53': 0,
 'p63': 0,
 'p73': 0}

### Phenotypes

In [7]:
outputs = ["Apoptosis", "CellCycleArrest","EMT","Invasion","Migration","Metastasis"]
def make_marker(*markers):
    for n in markers: # ensure nodes exist
        assert n in bn
    return {n: 1 if n in markers else 0 for n in outputs}
phenotypes = {
"HS": make_marker(),
"A+CA": make_marker("Apoptosis", "CellCycleArrest"),
"CA+EMT": make_marker("CellCycleArrest", "EMT"),
"Metastasis+CA": make_marker("CellCycleArrest","EMT","Invasion","Migration","Metastasis"),
"EMT+I": make_marker("EMT", "Invasion"),
"CA+EMT+I": make_marker("CellCycleArrest", "EMT", "Invasion")
}
phenotypes["HS"]["CDH1"] = 1
pd.DataFrame.from_dict(phenotypes, orient="index").fillna('')

,Apoptosis,CellCycleArrest,EMT,Invasion,Migration,Metastasis,CDH1
HS,0,0,0,0,0,0,1
A+CA,1,1,0,0,0,0,
CA+EMT,0,1,1,0,0,0,
Metastasis+CA,0,1,1,1,1,1,
EMT+I,0,0,1,1,0,0,
CA+EMT+I,0,1,1,1,0,0,


## Synthesis

In [8]:
data = {"init": init}
data.update(phenotypes)

In [9]:
bo = bonesis.BoNesis(dom, data)
bo.settings["parallel"] = threads

In [10]:
def has_cyclic(bn):
    mbn = mpbn.MPBooleanNetwork(bn)
    for a in mbn.attractors():
        if "*" in a.values():
            return True
    return False

In [11]:
def build_ensemble(bo, name, limit):
    zipfile = f"{bundle_prefix}{name}.zip"
    print(f"Building ensemble of size {limit} in {zipfile}")
    mylimit = 2*limit
    view = bo.diverse_boolean_networks(limit=mylimit, skip_supersets=True)
    with ZipFile(zipfile, "w") as bundle:
        n = 0
        for i, bn in enumerate(view):
            if exclude_cyclic and has_cyclic(bn):
                print("HAS CYCLIC ATTRACTORS, IGNORING")
                continue
            with bundle.open(f"bn{i}.bnet", "w") as fp:
                fp.write(bn.source().encode())
            n += 1
            if n == limit:
                break

### Wild-type constraints

#### v0

Each phenotype has at least one matching fixed point, which is reachable from at least one initial condition.

In [12]:
wt_phenotypes = ["HS", "A+CA", "CA+EMT", "Metastasis+CA"]

In [13]:
for ph in wt_phenotypes:
    # there is one configuration matching with init
    # which can reach a configuration being a fixed point and matching with ph
    +bo.obs("init") >= bo.fixed(+bo.obs(ph))

In [14]:
#%time build_ensemble(bo, "v0", limit)

#### globalfps

We add an universal constraint on fixed points: all of them should match with one of the phenotype.

In [15]:
bo.all_fixpoints({bo.obs(ph) for ph in wt_phenotypes});

In [16]:
#%time build_ensemble(bo, "globalfps", limit)

### Mutants

In [17]:
def mutant_constraint(mutation, phenotypes):
    with bo.mutant(mutation) as m:
        # each phenotype can be reached from at least one init
        for ph in phenotypes:
            +m.obs("init") >= m.fixed(+m.obs(ph))
        # each possible initial configuration can only reach fixed points matching with phenotypes
        for cfg in bonesis.matching_configurations(m.obs("init")):
            cfg >> "fixpoints" ^ {m.obs(ph) for ph in phenotypes}

In [ ]:
#mutant_constraint({"CTNNB1": 1}, ["CA+EMT+I", "EMT+I"])
mutant_constraint({"NICD": 1}, ["CA+EMT","CA+EMT+I","Metastasis+CA"])
mutant_constraint({"p53": 0}, wt_phenotypes)
%time build_ensemble(bo, "mutants-NICD-p53", limit)